![TELCO Logo](https://upload.wikimedia.org/wikipedia/commons/6/6d/Telco_System_Logo.jpg)

# TELCO Classification Project 
### - by Jeff Akins

## Executive Summary
#### Why are our customers churning?
- 